This block sets up a PDF and CDF that take aset of data (resids) measure, their mean, SD, skew and kurtosis and sets up the distributions

In [ ]:
x = np.linspace(stats.norm.ppf(0.01),
                stats.norm.ppf(0.99), 1000)

# this code sets up a distribution of residuals that reflects the empiric
# distrbiution of residuals from the regression, accounting for skew and kurotosis
# the following code trsnaltes this to a cumulative distribution function
# so that we can later sample from the CDF
pdfFunc = extras.pdf_mvsk([resids.mean(), resids.var(),
                           stats.skew(resids), stats.kurtosis(resids)])

pdfVals = pdfFunc(x)

# remove negative elements from pdf values, which can happen with large kurtosis
pdfVals = [val if val > 0 else 0 for val in pdfVals]
# normalize the PDF
normPDFVals = pdfVals / sum(pdfVals)

# get CDF from PDF
cdf = [0]
for idx, val in enumerate(normPDFVals):
    cdf.append(cdf[idx] + val)
del(cdf[0])

_residCDF = cdf

This is a method that gives you a random draw from the CDF above

In [ ]:

    def get_residual_draw_from_cdf(self):
        rand = np.random.random()
        distanceToCDF = pd.Series(data=[abs(val-rand)
                                        for val in self._residCDF],
                                  index=np.arange(0, len(self._residCDF)))
        minIndex = distanceToCDF.idxmin()
        return (float(minIndex) - len(self._residCDF)/2) / len(self._residCDF)